# MTH5 Example 07

## Make an MTH5 from Phoenix Data

This example demonstrates how to read Phoenix data into an MTH5 file.  These data are provided by Phoenix Geophysical Ltd. Provided are 3 synchronously recored stations.  The Phoenix instruments MTU-5 series are broadband instruments with multiple sample rates.  

The instrumnet is constantly recording at 24 kHz and depending on the input paramters those data are decimated or chunked into different data files.  A common setup is to record continuously at 150 samples per second and record short bursts of 24 kHz for just a few seconds every 15 minutes or so.  In this case the continuous 24 kHz is decimated down to 150 Hz and stored in files with a length of 6 minutes.  The short bursts are also stored into a separate file.  This is done for each channel recorded.  Its common for broadband instruments to record each channel individually making it easier to buffer.  

A challenge arises when you have the short bursts is that you will have a ton of runs.  Luckily HDF5 can handle this, we just have to organize it properly.  And figure out which file belongs where.  A `PhoenixCollection` object is provided for just this task. 

## Imports

In [2]:
from pathlib import Path

from mth5.mth5 import MTH5
from mth5 import read_file
from mth5.io.phoenix import ReceiverMetadataJSON, PhoenixCollection

2022-09-27 12:11:26,772 [line 135] mth5.setup_logger - INFO: Logging file can be found C:\Users\jpeacock\OneDrive - DOI\Documents\GitHub\mth5\logs\mth5_debug.log


## Data Directory

Specify the station directory.  Phoenix files place each channel in a folder under the station directory named by the channel number.  There is also a `recmeta.json` file that has metadata output by the receiver that can be useful.  In the `PhoenixGeopPy/sample_data` there are 2 folders one for native data, these are `.bin` files which are the raw data in counts sampled at 24k.  There is also a folder for segmented files, these files are calibrated to millivolts and decimated or segmented data according to the recording configuration.  Most of the time you would use the segmented files? 

In [5]:
station_dir = Path().cwd().parent.parent.joinpath("data", "time_series", "phoenix")

## File Collection

We've developed a collection dataframe to help sort out which files are which and which files can be grouped together into runs.  Continous runs will be given a single run name and segmented data will have sequential run names.  Both will have the pattern `sr{sample_rate}_####` for the `run.id`.

Here `PhoenixCollection.get_runs` returns a two level ordered dictionary (`OrderedDict`).  The first level is keyed by station ID.  These objects are in turn ordered dictionaries by run ID.  Therefore you can loop over stations and runs.

#### Receiver Metadata

The data logger or receiver will output a `JSON` file that contains useful metadata that is missing from the data files.  The `recmeta.json` file can be read into an object with methods to translate to `mt_metadata` objects. This is read in by `PhoenixCollection` and is in the attribute `receiver_metadata`. 


In [6]:
phx_collection = PhoenixCollection(file_path=station_dir)
run_dict = phx_collection.get_runs(sample_rates=[150, 24000])

## Initiate MTH5

First initiate an MTH5 file, can use the receiver metadata to fill in some `Survey` metadata

In [7]:
m = MTH5()
m.open_mth5(station_dir.joinpath("mth5_from_phoenix.h5"), "w")

2022-09-27 14:18:48,869 [line 672] mth5.mth5.MTH5._initialize_file - INFO: Initialized MTH5 0.2.0 file C:\Users\jpeacock\OneDrive - DOI\Documents\GitHub\mt_examples\data\time_series\phoenix\mth5_from_phoenix.h5 in mode w


## Loop through Surveys, Stations, Runs, and Channels

Using the `run_dict` value output by the `PhoenixCollection.get_runs` we can simply loop through the runs without knowing if the data are continous or discontinuous, the `read_file` will take care of that.

Users should note the Phoenix file structure.  Inside the folder are files with extensions of `.td_24k` and `td_150`.  

- `.td_24k` are usually bursts of a few seconds of data sampled at 24k samples per second to get high frequency information.  The returned object is a `mth5.timeseries.ChannelTS`.
- `td_150` is data continuously sampled at 150 samples per second.  These files usually have a set length, commonly an hour. The returned object is a `mth5.timeseries.ChannelTS`.



In [10]:
%%time

for station_id, station_dict in run_dict.items():
    
    phx_metadata = phx_collection.metadata_dict[station_id]
    
    survey_group = m.add_survey(
        phx_metadata.survey_metadata.id,
        survey_metadata=phx_metadata.survey_metadata)
    
    station_metadata = phx_metadata.station_metadata
    station_group = survey_group.stations_group.add_station(
        station_metadata.id, 
        station_metadata=station_metadata
    )
    for run_id, run_df in station_dict.items():
        run_metadata = phx_metadata.run_metadata
        run_metadata.id = run_id
        run_metadata.sample_rate = float(run_df.sample_rate.unique()[0])

        run_group = station_group.add_run(run_metadata.id, run_metadata=run_metadata)
        for row in run_df.itertuples():
            ch_ts = read_file(row.fn, **{"channel_map":phx_metadata.channel_map})
            ch_metadata = phx_metadata.get_ch_metadata(
                ch_ts.channel_metadata.channel_number
            )
            # need to update the time period and sample rate as estimated from the data not the metadata
            ch_metadata.sample_rate = ch_ts.sample_rate
            ch_metadata.time_period.update(ch_ts.channel_metadata.time_period)
            ch_ts.channel_metadata.update(ch_metadata)

            # add channel to the run group
            ch_dataset = run_group.from_channel_ts(ch_ts)
            
        # update run metadata with information from the input channels
        run_group.validate_run_metadata()
        
    # update the station metadata based on new runs
    station_group.validate_station_metadata()
    
    # update survey metadata based on new station
    survey_group.update_survey_metadata()
        

2022-09-27 14:22:24,943 [line 285] mth5.groups.base.MasterSurvey.add_survey - INFO: survey SU already exists, returning existing group.
2022-09-27 14:22:46,986 [line 285] mth5.groups.base.MasterSurvey.add_survey - INFO: survey UofS already exists, returning existing group.
2022-09-27 14:23:21,883 [line 285] mth5.groups.base.MasterSurvey.add_survey - INFO: survey UofS already exists, returning existing group.
2022-09-27 14:25:16,186 [line 285] mth5.groups.base.MasterSurvey.add_survey - INFO: survey UofS already exists, returning existing group.
2022-09-27 14:25:45,898 [line 285] mth5.groups.base.MasterSurvey.add_survey - INFO: survey UofS already exists, returning existing group.
2022-09-27 14:26:55,595 [line 285] mth5.groups.base.MasterSurvey.add_survey - INFO: survey UofS already exists, returning existing group.
2022-09-27 14:28:10,842 [line 285] mth5.groups.base.MasterSurvey.add_survey - INFO: survey UofS already exists, returning existing group.


Wall time: 11min 39s


In [12]:
m.channel_summary.summarize()
m.channel_summary.to_dataframe()

,survey,station,run,latitude,longitude,elevation,component,start,end,n_samples,sample_rate,measurement_type,azimuth,tilt,units,hdf5_reference,run_hdf5_reference,station_hdf5_reference
0,SU,Elbow,sr150_0001,50.979812,-106.736059,568.883806,e1,2019-09-03 21:24:04+00:00,2019-09-04 08:09:48+00:00,5811600,150.0,electric,0.0,0.0,millivolts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
1,SU,Elbow,sr150_0001,50.979812,-106.736059,568.883806,e2,2019-09-03 21:24:04+00:00,2019-09-04 08:09:48+00:00,5811600,150.0,electric,0.0,0.0,millivolts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
2,SU,Elbow,sr150_0001,50.979812,-106.736059,568.883806,h1,2019-09-03 21:24:04+00:00,2019-09-04 08:09:48+00:00,5811600,150.0,magnetic,0.0,0.0,millivolts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
3,SU,Elbow,sr150_0001,50.979812,-106.736059,568.883806,h2,2019-09-03 21:24:04+00:00,2019-09-04 08:09:48+00:00,5811600,150.0,magnetic,0.0,0.0,millivolts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
4,SU,Elbow,sr150_0001,50.979812,-106.736059,568.883806,h3,2019-09-03 21:24:04+00:00,2019-09-04 08:09:48+00:00,5811600,150.0,magnetic,0.0,0.0,millivolts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4677,UofS,REMOTE_03,sr24k_0124,50.979666,-106.794989,559.472154,h2,2019-09-06 14:14:31+00:00,2019-09-06 14:14:33+00:00,48000,24000.0,magnetic,0.0,0.0,millivolts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
4678,UofS,REMOTE_03,sr24k_0124,50.979666,-106.794989,559.472154,h3,2019-09-06 14:14:31+00:00,2019-09-06 14:14:33+00:00,48000,24000.0,magnetic,0.0,0.0,millivolts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
4679,UofS,REMOTE_03,sr24k_0124,50.979666,-106.794989,559.472154,h4,2019-09-06 14:14:31+00:00,2019-09-06 14:14:33+00:00,48000,24000.0,magnetic,0.0,0.0,millivolts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>
4680,UofS,REMOTE_03,sr24k_0124,50.979666,-106.794989,559.472154,h5,2019-09-06 14:14:31+00:00,2019-09-06 14:14:33+00:00,48000,24000.0,magnetic,0.0,0.0,millivolts,<HDF5 object reference>,<HDF5 object reference>,<HDF5 object reference>


In [13]:
m.close_mth5()

2022-09-27 15:02:28,403 [line 753] mth5.mth5.MTH5.close_mth5 - INFO: Flushing and closing C:\Users\jpeacock\OneDrive - DOI\Documents\GitHub\mt_examples\data\time_series\phoenix\mth5_from_phoenix.h5
